# Importando e tratando a base de dados

><h1>Importação dos dados

In [ ]:
import json
import requests
from io import StringIO

# carregando as credenciais
with open("/home/daniel/geohmount/code/credentials.json", "r") as credentials_file:
    credentials = json.load(credentials_file)
# parametros da request (github info)
owner = credentials["github"]["username"]
token = credentials["github"]["token"]
repo = credentials["github"]["repo"]
path = 'trajs/sb/resultado.csv'

# enviando o request
r = requests.get(f'https://api.github.com/repos/{owner}/{repo}/contents/{path}',
    headers={
        'accept': 'application/vnd.github.v3.raw',
        'authorization': f'token {token}'
            }
    )

# convertendo a string em objeto stringIO
string_io_obj = StringIO(r.text)

In [ ]:
import pandas as pd

trajs = pd.read_csv(string_io_obj, sep=";")

In [ ]:
trajs.head(15)

><h1>Tratamento dos dados

>><h2>Renomeando colunas

In [ ]:
trajs = trajs.rename(columns={'data': 'data_full', 'velocidade_km/h': 'velocidade'})

>><h2>Removendo trajetórias sem campanha

In [ ]:
trajs = trajs.dropna()
trajs

>><h2>Criando a coluna categórica dos sentidos das trajetórias

In [ ]:
sentidos_switcher = {0: "N", # dicionário com os sentidos de acordo com a rosa dos ventos
        1: "NNE",
        2: "NE",
        3: "ENE",
        4: "E",
        5: "ESE",
        6: "SE",
        7: "SSE",
        8: "S",
        9: "SSW",
        10: "SW",
        11: "WSW",
        12:"W",
        13:"WNW",
        14:"NW",
        15:"NNW",
}
sentidos = list() #lista que vai armazenar os sentidos
for value in trajs['direcao_graus']: 
  quociente = value // 22.5
  resto = value % 22.5              # laço de repetição que atribui um número inteiro de acordo com cada direção em graus
  if resto > 11.25: 
    quociente += 1
  sentidos.append(sentidos_switcher.get(quociente))
trajs['direcao_cat'] = sentidos
trajs.head(10)

>><h2>Criando a coluna categórica de velocidades

In [ ]:
velocidades_switcher = {0: '0-5 km/h', #dicionário com as velocidades categóricas
               1: '5-10 km/h',
               2: '10-15 km/h',
               3: '15-20 km/h',
               4: '20-25 km/h',
               5: '25-30 km/h',
               6: '30-35 km/h',
               7: '35-40 km/h',
               8: '40-45 km/h',
}
velocidades = list()
for value in trajs['velocidade']:
  quociente = value // 5
  velocidades.append(velocidades_switcher.get(quociente))
trajs['velocidade_cat'] = velocidades
trajs.head(10)

>><h2>Criando a coluna categórica de soma de chuva do evento (dados da estação meteorológica)

In [ ]:
chuvas_switcher = {0: '5-20 mm', #dicionário com as chuvas categóricas
               1: '20-35 mm',
               2: '35-50 mm',
               3: '50-65 mm',
               4: '65-80 mm',
               5: '80-95 mm',
               6: '95-110 mm',
               7: '110-125 mm',
               8: '125-130 mm',
}
chuvas = list()
for value in trajs['evt_sum_rain']:
  value -= 5
  quociente = value // 15
  chuvas.append(chuvas_switcher.get(quociente))
trajs['evt_sum_rain_cat'] = chuvas
trajs.head(10)

>><h2>Criando a coluna categórica de data formatada em 'mês, ano'

In [ ]:
trajs['data'] = pd.to_datetime(trajs['data_full'])
trajs['data'] = trajs['data'].dt.strftime('%B, %Y')
trajs.head()

>><h2>Criando datasets separados por altitude

In [ ]:
trajs_509 = trajs.query("altitude == 509")
trajs_659 = trajs.query("altitude == 659")
trajs_909 = trajs.query("altitude == 909")

# Criando os gráficos gerais

><h2> Velocidade

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

velocidades = ['0-5 km/h', '5-10 km/h', '10-15 km/h', '15-20 km/h', '20-25 km/h', '25-30 km/h', '30-35 km/h', '35-40 km/h','40-45 km/h']
direcoes = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]

fig = px.bar_polar(trajs_659, r='velocidade', theta='direcao_cat', color='velocidade_cat', 
                   category_orders={'velocidade_cat': velocidades,
                                    'direcao_cat': direcoes}, 
                   color_discrete_sequence = px.colors.sequential.Plasma_r,)

fig.update_layout(legend=dict(
     traceorder='reversed',
     yanchor='top',
     y=0.99,
     xanchor='right',
     x=1.1,
     title_font_size=14),
     legend_title_text='<b>Velocidade',
     title={'text': '<b>Distribuição das massas de ar resultantes em chuva no Soberbo',
            'y': 0.98, 'x': 0.5, 'xref': 'paper'},
     font_family='Open Sans',
     polar_radialaxis_showticklabels=False,
     annotations=[dict(
        showarrow=False,
        xref='paper',
        yref='paper',
        x=0.01, y=0.95,
        text='<b>Altitude: 659m',
        font=dict(size=14, color='black'))],
     polar_angularaxis_tickfont_size = 15,
     paper_bgcolor='rgb(255, 255, 255)'
)

fig.update_traces(hovertemplate=
                  '<b>%{theta}</b><br>'+
                  'velocidade (km/h): %{r:.1f}<br>')

fig.show()

><h2> Chuva

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

chuvas = ['5-20 mm', '20-35 mm', '35-50 mm', '50-65 mm', '65-80 mm', '80-95 mm', '95-110 mm', '110-125 mm', '125-130 mm']
direcoes = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]

fig = px.bar_polar(trajs_659, r='evt_sum_rain', theta='direcao_cat', color='evt_sum_rain_cat', 
                   category_orders={'evt_sum_rain_cat': chuvas,
                                    'direcao_cat': direcoes}, 
                   color_discrete_sequence = px.colors.sequential.Blues,)

fig.update_layout(legend=dict(
     traceorder='reversed',
     yanchor='top',
     y=0.99,
     xanchor='right',
     x=1.1,
     title_font_size=14),
     legend_title_text='<b>Precipitação',
     title={'text': '<b>Distribuição das massas de ar resultantes em chuva no Soberbo',
            'y': 0.98, 'x': 0.5, 'xref': 'paper'},
     font_family='Open Sans',
     polar_radialaxis_showticklabels=False,
     annotations=[dict(
        showarrow=False,
        xref='paper',
        yref='paper',
        x=0.01, y=0.95,
        text='<b>Altitude: 659m',
        font=dict(size=14, color='black'))],
     polar_angularaxis_tickfont_size = 15,
     paper_bgcolor='rgb(255, 255, 255)'
)

fig.update_traces(hovertemplate=
                  '<b>%{theta}</b><br>'+
                  'chuva (mm): %{r:.1f}<br>')

fig.show()

# Criando os gráficos por campanha

><h2>Velocidade

In [ ]:
import plotly.express as px

velocidades = ['0-5 km/h', '5-10 km/h', '10-15 km/h', '15-20 km/h', '20-25 km/h', '25-30 km/h', '30-35 km/h', '35-40 km/h','40-45 km/h']
direcoes = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]

campanhas = trajs_659['campanha'].unique()

# padronizar as cores nos gráficos pelo valor de velocidade_cat
color_discrete_map = {'0-5 km/h': 'rgb(240, 249, 33)', '5-10 km/h': 'rgb(253, 202, 38)', '10-15 km/h': 'rgb(251, 159, 58)', '15-20 km/h': 'rgb(237, 121, 83)', '20-25 km/h': 'rgb(216, 87, 107)', '25-30 km/h': 'rgb(189, 55, 134)', '30-35 km/h': 'rgb(156, 23, 158)', '35-40 km/h': 'rgb(114, 1, 168)','40-45 km/h': 'rgb(70, 3, 159)'}

for campanha in campanhas:
    df = trajs_659.loc[trajs_659["campanha"] == campanha]
        
    fig = px.bar_polar(df, r='velocidade', theta='direcao_cat', color='velocidade_cat', color_discrete_map=color_discrete_map,
                      category_orders={'velocidade_cat': velocidades,
                                      'direcao_cat': direcoes}, 
                       color_discrete_sequence = px.colors.sequential.Plasma_r)

    fig.update_layout(legend=dict(
         traceorder='reversed',
         yanchor='top',
         y=0.99,
         xanchor='right',
         x=1.1,
         title=dict(
             text='<b>Velocidade', 
             font={'size': 14})),
         title=f'<b>{campanha}</b>',
         font_family='Open Sans',
         polar_radialaxis_showticklabels=False,
         polar_angularaxis_tickfont_size = 15,
         paper_bgcolor='rgb(255, 255, 255)')

    fig.show()

><h2>Chuva

In [ ]:
import plotly.express as px

chuvas = ['5-20 mm', '20-35 mm', '35-50 mm', '50-65 mm', '65-80 mm', '80-95 mm', '95-110 mm', '110-125 mm', '125-130 mm']
direcoes = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]

campanhas = trajs_659['campanha'].unique()

# padronizar as cores nos gráficos pelo valor de evt_sum_rain_cat
color_discrete_map = {'5-20 mm': 'rgb(247,251,255)', '20-35 mm': 'rgb(222,235,247)', '35-50 mm': 'rgb(198,219,239)', '50-65 mm': 'rgb(158,202,225)', '65-80 mm': 'rgb(107,174,214)', '80-95 mm': 'rgb(66,146,198)', '95-110 mm': 'rgb(33,113,181)', '110-125 mm': 'rgb(8,81,156)', '125-130 mm': 'rgb(8,48,107)'}

for campanha in campanhas:
    df = trajs_659.loc[trajs_659["campanha"] == campanha]
        
    fig = px.bar_polar(df, r='evt_sum_rain', theta='direcao_cat', color='evt_sum_rain_cat', color_discrete_map=color_discrete_map,
                      category_orders={'evt_sum_rain_cat': chuvas,
                                      'direcao_cat': direcoes}, 
                       color_discrete_sequence = px.colors.sequential.Blues)

    fig.update_layout(legend=dict(
         traceorder='reversed',
         yanchor='top',
         y=0.99,
         xanchor='right',
         x=1.1,
         title=dict(
             text='<b>Precipitação', 
             font={'size': 14})),
         title=f'<b>{campanha}</b>',
         font_family='Open Sans',
         polar_radialaxis_showticklabels=False,
         polar_angularaxis_tickfont_size = 15,
         template='plotly_dark')
    
    fig.update_polars()

    fig.show()